## Day 19 - Some dodgy workflows

In [1]:
with open("./example.txt") as f:
    example_inputs = f.read().strip().split("\n\n")

with open("./input.txt") as f:
    input_inputs = f.read().strip().split("\n\n")

example_inputs

['px{a<2006:qkq,m>2090:A,rfg}\npv{a>1716:R,A}\nlnx{m>1548:A,A}\nrfg{s<537:gd,x>2440:R,A}\nqs{s>3448:A,lnx}\nqkq{x<1416:A,crn}\ncrn{x>2662:A,R}\nin{s<1351:px,qqz}\nqqz{s>2770:qs,m<1801:hdj,R}\ngd{a>3333:R,R}\nhdj{m>838:A,pv}',
 '{x=787,m=2655,a=1222,s=2876}\n{x=1679,m=44,a=2067,s=496}\n{x=2036,m=264,a=79,s=2244}\n{x=2461,m=1339,a=466,s=291}\n{x=2127,m=1623,a=2188,s=1013}']

In [2]:
def part1(inputs: list[str]) -> int:
    """
    Return sum of the parts for each accepted item
    """

    workflows_raw, parts_raw = inputs

    workflows = workflows_raw.split()
    workflow_map = {}

    def extract_func(contents: str) -> callable:
        def f(x: int, m: int, a: int, s: int):
            for content in contents:
                if ":" in content:
                    comparison_str, mapped_to = content.split(":")
                    if eval(comparison_str):
                        # this will use one of x, m, a, s
                        return mapped_to
                else:
                    return content
            
            raise RuntimeError("Didn't leave the generated workflow function, panic!")
        
        return f


    for workflow_str in workflows:
        name = workflow_str.split("{")[0]
        contents = (workflow_str[:-1].split("{")[1]).split(",")         
        workflow_map[name] = extract_func(contents)
    

    parts_strings = parts_raw.split("\n")
    parts_strings = [part_string[1:-1] for part_string in parts_strings]
    
    accepted_sum = 0
    for parts_str in parts_strings:
        args = [
            int(p.split("=")[-1])
            for p in parts_str.split(",")
            ]
        at_loc = "in"
        while at_loc not in ("A", "R"):
            at_loc = workflow_map[at_loc](*args)
        
        if at_loc == "A":
            accepted_sum += sum(args)
    
    return accepted_sum

assert part1(example_inputs) == 19114
part1(input_inputs)

399284

**Part 2: lots more to do lol**


Each of the four ratings (x, m, a, s) can have an integer value ranging from a minimum of 1 to a maximum of 4000. Of all possible distinct combinations of ratings, your job is to figure out which ones will be accepted.

In [4]:
4000*4000*4000*4000

256000000000000

In [17]:
pow(167409079868000, 0.25)

3597.035734825834

In [16]:
from collections import deque

def part2(inputs: list[str]) -> int:
    workflows_raw, _ = inputs

    workflows = workflows_raw.split()
    workflow_map = {}

    def extract_path(contents: str) -> callable:
        path = []
        for content in contents:
            if ":" in content:
                comparison_str, mapped_to = content.split(":")
                path.append((comparison_str, mapped_to))
            else:
                mapped_to = content
                path.append((None, mapped_to))

        to_mappings = set(to_map[1] if isinstance(to_map, tuple) else to_map for to_map in path)

        if len(to_mappings) == 1:
            return to_mappings.pop()
        return path


    for workflow_str in workflows:
        name = workflow_str.split("{")[0]
        contents = (workflow_str[:-1].split("{")[1]).split(",")      
        workflow_map[name] = extract_path(contents)

    workflow_map["A"] = "A"
    workflow_map["R"] = "R"

    x_range = (1,4000)
    m_range = (1,4000)
    a_range = (1,4000)
    s_range = (1,4000)

    # LETS TAKE ALL THE PATHS THROUGH THE BLOODY MAZE, LIKE A QUEUE WE POP
    # while queue , type of idea
    # and chop down the ranges as we go but this time we're parsing through comparisons
    
    current_loc = "in"
    queue = deque([(current_loc, x_range, m_range, a_range, s_range)])

    ranges_that_satisfy = []
    while queue:
        # xrange, mrange, arange, srange
        current_loc, (xl, xu), (ml, mu), (al, au), (sl, su) = queue.popleft()
        next_loc = workflow_map[current_loc]

        if isinstance(next_loc, str):
            # directly to next location!
            if next_loc == "A":
                ranges_that_satisfy.append(
                    ((xl, xu), (ml, mu), (al, au), (sl, su))
                )
            elif next_loc == "R":
                pass
            else:
                queue.append(
                    (next_loc, (xl, xu), (ml, mu), (al, au), (sl, su))
                )
        else:
            assert isinstance(next_loc, list)
            # this is the case with instructions...
            # e.g. [('a<2006', 'qkq'), ('m>2090', 'A'), (None, 'rfg')]
            for comparison, loc_ in next_loc:
                if comparison is None:
                    # e.g. (None, 'rfg') means straight to rfg
                    if loc_ == "A":
                        ranges_that_satisfy.append(
                            ((xl, xu), (ml, mu), (al, au), (sl, su))
                        )
                    elif loc_ == "R":
                        continue
                    else:
                        queue.append(
                            (loc_, (xl, xu), (ml, mu), (al, au), (sl, su))
                        )
                else:
                    # e.g. ('a<2006', 'qkq')
                    comparison_char = comparison[0]
                    if comparison_char == "x":
                        # lower case
                        x = xl
                        lower_passes = eval(comparison)
                        # upper case
                        x = xu
                        upper_passes = eval(comparison)

                        if upper_passes and lower_passes:
                            queue.append(
                                (loc_, (xl, xu), (ml, mu), (al, au), (sl, su))
                            )
                        # append to queue the passing range, then
                        # change xl, xu for the next iteration of this loop (i.e. for the non passing route)
                        else:
                            comparison_value = int(comparison[2:])
                            if upper_passes and not lower_passes:
                                # means it was a >
                                queue.append(
                                    (loc_, (comparison_value + 1, xu), (ml, mu), (al, au), (sl, su))
                                )
                                xu = comparison_value
                            elif lower_passes and not upper_passes:
                                # means it was a <
                                queue.append(
                                    (loc_, (xl, comparison_value - 1), (ml, mu), (al, au), (sl, su))
                                )
                                xl = comparison_value
                            elif not lower_passes and not upper_passes:
                                # just go to the next
                                continue
                            else:
                                raise ValueError("How did we reach here>>>")
                    elif comparison_char == "m":
                        # lower case
                        m = ml
                        lower_passes = eval(comparison)
                        # upper case
                        m = mu
                        upper_passes = eval(comparison)

                        if upper_passes and lower_passes:
                            queue.append(
                                (loc_, (xl, xu), (ml, mu), (al, au), (sl, su))
                            )
                        # append to queue the passing range, then change range for next loop
                        else:
                            comparison_value = int(comparison[2:])
                            if upper_passes and not lower_passes:
                                # means it was a >
                                queue.append(
                                    (loc_, (xl, xu), (comparison_value + 1, mu), (al, au), (sl, su))
                                )
                                mu = comparison_value
                            elif lower_passes and not upper_passes:
                                # means it was a <
                                queue.append(
                                    (loc_, (xl, xu), (ml, comparison_value - 1), (al, au), (sl, su))
                                )
                                ml = comparison_value
                            elif not lower_passes and not upper_passes:
                                # just go to the next
                                continue
                            else:
                                raise ValueError("How did we reach here>>>")
                    elif comparison_char == "a":
                        # lower case
                        a = al
                        lower_passes = eval(comparison)
                        # upper case
                        a = au
                        upper_passes = eval(comparison)

                        if upper_passes and lower_passes:
                            queue.append(
                                (loc_, (xl, xu), (ml, mu), (al, au), (sl, su))
                            )
                        # append to queue the passing range, then change range for next loop
                        else:
                            comparison_value = int(comparison[2:])
                            if upper_passes and not lower_passes:
                                # means it was a >
                                queue.append(
                                    (loc_, (xl, xu), (ml, mu), (comparison_value + 1, au), (sl, su))
                                )
                                au = comparison_value
                            elif lower_passes and not upper_passes:
                                # means it was a <
                                queue.append(
                                    (loc_, (xl, xu), (ml, mu), (al, comparison_value - 1), (sl, su))
                                )
                                al = comparison_value
                            elif not lower_passes and not upper_passes:
                                # just go to the next
                                continue
                            else:
                                raise ValueError("How did we reach here>>>")
                    elif comparison_char == "s":
                        # lower case
                        s = sl
                        lower_passes = eval(comparison)
                        # upper case
                        s = su
                        upper_passes = eval(comparison)

                        if upper_passes and lower_passes:
                            queue.append(
                                (loc_, (xl, xu), (ml, mu), (al, au), (sl, su))
                            )
                        # append to queue the passing range, then change range for next loop
                        else:
                            comparison_value = int(comparison[2:])
                            if upper_passes and not lower_passes:
                                # means it was a >
                                queue.append(
                                    (loc_, (xl, xu), (ml, mu), (al, au), (comparison_value + 1, su))
                                )
                                su = comparison_value
                            elif lower_passes and not upper_passes:
                                # means it was a <
                                queue.append(
                                    (loc_, (xl, xu), (ml, mu), (al, au), (sl, comparison_value - 1))
                                )
                                sl = comparison_value
                            elif not lower_passes and not upper_passes:
                                # just go to the next
                                continue
                            else:
                                raise ValueError("How did we reach here>>>")
    

    amount_possible = 0
    for (xl, xu), (ml, mu), (al, au), (sl, su) in ranges_that_satisfy:
        amount_possible += (
            (xu-xl+1)*(mu-ml+1)*(au-al+1)*(su-sl+1)
        )
        

    return amount_possible

part2(example_inputs) == 167409079868000
part2(input_inputs)

121964982771486